In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        def extraction():
            rnd.shuffle(whole_numbers)
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return [extraction() for _ in range(nums)]

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        return rnd.sample(whole_numbers, k=nums)

    def int(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return combo

    @staticmethod
    def manyMethod(backend):
        def inner(nums, max_nums):
            samples = [backend(nums, max_nums) for _ in range(max_nums)]
            rnd.shuffle(samples)

            return rnd.choice(samples)

        return inner

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        backend = self.manyMethod(backend) if self.many else backend
        combo = backend(self.numbers, self.max_numbers)

        if self.extra:
            extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri Estratti:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(extra))  # type: ignore

In [2]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [3]:
superenalotto(backend='choice')

Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 5 14 27 67 76 82


In [4]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [5]:
eurojack()

Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 21 23 32 47 49
Superstar: 1 2


In [6]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 7 14 21 22 47
Superstar: 2 8
backend: choice
######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 1 2 6 36 49
Superstar: 1 10
backend: int
######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 6 15 32 35 45
Superstar: 5 7
backend: sample

################# many ####################

######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 1 4 21 27 40
Superstar: 6 11
backend: choice
######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 12 17 24 40 43
Superstar: 4 11
backend: int
######
Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 17 28 30 48 49
Superstar: 7 8
backend: sample


In [7]:
superenalotto.extra=1
superenalotto(backend='choice', many=False)

Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 26 29 33 48 50 83
Superstar: 36


In [8]:
superenalotto(backend='int')

Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 29 41 54 56 75 89
Superstar: 12


In [9]:
superenalotto.extra

1

In [10]:
superenalotto.many

True

In [11]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 05/06/2023 10:50 
Numeri Estratti: 1 4 6 7 10 11 16 17 19 20
Superstar: 11


In [12]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter